### RAG(검색 증강 생성) 파이프라인 성능 평가

1. 데이터셋 구축
2. 모델 평가
3. 평가 결과 시각화
4. 평가 결과 분석
5. 평가 결과 개선
---

### RAG(Retrieval Augmented Generation) 평가용 데이터셋 구축

RAG를 평가할때는 Question, Answer, Retrieval 3가지 정보를 일반적으로 필요로 함
* Question : 사용자의 질문
* Answer : 모델의 답변
* Retrieval : 모델이 사용한 문서 정보

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
!pip install -q pandas

In [16]:
import pandas as pd

# 질문, 답변 목록
inputs = [
    "백설공주는 왜 백설공주라는 이름을 가지게 되었나요?",
    "백설공주를 죽이려고 했던 사람은 누구인가요?",
    "백설공주는 어떤 음식을 먹고 쓰러졌나요?",
]

outputs = [
    "백설공주는 피부가 눈처럼 하얗고 아름다워서 백설공주라는 이름을 가지게 되었어요",
    "백설공주의 새어머니인 왕비가 백설공주를 죽이려고 했어요. 왕비는 자신이 세상에서 가장 아름다운 사람이 되고싶어서 백설공주를 질투했기 때문이에요",
    "백설공주는 빨간 독사과를 먹고 쓰러졌어요.",
]

# 답변 쌍으로 만들기
# zip() : 파이썬 내장함수 여러개의 반복가능한 객체들을 병령적으로 묶어주는 역할
qa_paris = [{"question" : q, "answer" : a} for q,a in zip(inputs,outputs)]

df = pd.DataFrame(qa_paris)

df.head()

,question,answer
0,백설공주는 왜 백설공주라는 이름을 가지게 되었나요?,백설공주는 피부가 눈처럼 하얗고 아름다워서 백설공주라는 이름을 가지게 되었어요
1,백설공주를 죽이려고 했던 사람은 누구인가요?,백설공주의 새어머니인 왕비가 백설공주를 죽이려고 했어요. 왕비는 자신이 세상에서 가...
2,백설공주는 어떤 음식을 먹고 쓰러졌나요?,백설공주는 빨간 독사과를 먹고 쓰러졌어요.


In [8]:
from langsmith import Client

client = Client()
dataset_name = "RAG_EVALUATION_DATASET"

def create_dataset(client, dataset_name, description=None):
    
    # 기존의 모든 데이터셋을 순회
    for dataset in client.list_datasets():
        # 동일한 이름의 데이터셋이 이미 존재하면 기존 데이터셋 반환
        if dataset.name == dataset_name:
            return dataset
        
    # 동일 이름의 데이터셋이 없으면 새로운 데이터셋 생성
    dataset = client.create_dataset(
        dataset_name = dataset_name, # 데이터셋 이름 설정
        description = description     # 데이터셋 설명 설정
    )
    
    return dataset

In [13]:
# 데이터셋 생성

dataset = create_dataset(client, dataset_name)

client.create_examples(
    inputs=[{"question" : q} for q in df["question"].tolist()],
    outputs= [{"answer": a} for a in df["answer"].tolist()],
    dataset_id=dataset.id
)

In [15]:
new_question = [
    "백설공주는 어떤 과일은 먹고 쓰러졌나요?",
    "백설공주는 왜 숲속으로 도망갔나요?"
]

new_answer = [
    "백설공주는 오렌지를 먹고 쓰러졌어요. 새엄마가 독을 넣은 오렌지를 건내주었죠",
    "백설공주는 학교 친구들과 싸워서 화가나서 숲속으로 도망갔어요"
]

client.create_examples(
    inputs = [{"question" : q} for q in new_question],
    outputs = [{"answer":a} for a in new_answer],
    dataset_id= dataset.id
)